<a href="https://colab.research.google.com/github/EvgenyKrylov78/credit_score/blob/main/N_book_%D0%BF%D1%80%D0%B8%D0%BD%D1%8F%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Review

Re: Привет, Роман!

    Отправляю исправленное.
    Спасибо за поддержку!


Привет, Евгений. Меня зовут Роман, я буду проверять твой проект.

Если не против, я буду вести общение "на ты".

Ты сможешь найти мои комментарии к коду в разноцветных ячейках как эти:

<div class="alert alert-success">
    Если все отлично
</div>

<div class="alert alert-warning">
    Если у меня есть некие предложения и рекомендации
</div>

<div class="alert alert-danger">
    Если присутствуют неточности, которые следует исправить
</div>

Пожалуйста, не удаляй мои коментарии. Если тебе захочется в последующем оставить свои коментарии на мои замечания, то ты можешь их **<font color="orange">выделить цветом</font>**  или **жирным шрифтом** , так будет проще отследить все наши с тобой правки в проекте.

# <font color="orange">Результат ревью:</font>
Прежде всего спасибо что выполнил все части проекта и прислал его нам.  
Ты проделал большую работу!  
Не смотря на то, что в в проекте есть несколько красных комментариев означающих, что там есть что поправить, твоя работа, на самом деле, выглядит достойно!  
Уверен, что тебе удастся быстро поправить проект, и я обязательно его приму.  
Желаю удачи!


# <font color="orange">Результат ревью 2:</font>
Евгений, привет!  
Проект принят. Поздравляю!  
Желаю удачи в наступившем 21 году и успехов на следующих спринтах.

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Изучение общей информации. 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/datasets/data.csv')

In [ ]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<div class="alert alert-warning">

### Комментарий ревьюера

смотреть на табличку удобнее так

        df.head()
    
или так

        display(df.head())
</div>

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.describe().astype(int)

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525,19351,21525,21525,21525,21525,19351
mean,0,63046,43,0,0,0,167422
std,1,140827,12,0,1,0,102971
min,-1,-18388,0,0,0,0,20667
25%,0,-2747,33,1,0,0,103053
50%,0,-1203,42,1,0,0,145017
75%,1,-291,53,1,1,0,203435
max,20,401755,75,4,4,1,2265604


Исследуем содержание, типы данных и факт заполнения важных для анализа столбцов: debt, children, family_status, total_income, purpose.

1. Пропущены значения total_income (доход).
2. children обратить внимание на минимальное значение "-1" и максимальное "20".
3. dob_years возраст не может быть "0" - обратить внимание. 
4. days_employed: а) отрицательное количество дней трудового стажа  (эти данные не участвуют в текущем анализе); б) Также макимальное значение выхоодит за пределы разумного (проанализировать вехний диапазон, данные должны быть в годах). Причиной является ошибка заполнения, надо понять на что поменять; в) пустоты - чем заполнить?. 

In [ ]:
'Значения по состоянию задолженности', df['debt'].unique()

('Значения по состоянию задолженности', array([0, 1]))

Значние "1" соответствует просроченным обязательствам. 

In [ ]:
'Просрочников: {:.1%}'.format(df['debt'].mean())

'Просрочников: 8.1%'

In [ ]:
('Значения по кол-ву детей у заемщика',
df['children'].unique())

('Значения по кол-ву детей у заемщика',
 array([ 1,  0,  3,  2, -1,  4, 20,  5]))

Артефакт: значения должны быть >=0, понять на что менять

In [ ]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Минус 1 значение заменю на 1 (предположу, что введен лишний знак), <font color="orange">остальные значения остаются как есть</font>

<div class="alert alert-danger">

### Комментарий ревьюера

Нет, для банка важно именно количество. Между семьей с 1 ребенком и с 3-мя детьми - целая финансовая пропасть
</div>

<div class="alert alert-success">

### Комментарий ревьюера 2

Супер
</div>

In [ ]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Данные в порядке - пять чётких групп

In [ ]:
df['total_income'].unique()

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

Требуется группировка после заполнения пропусков

In [ ]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Требуется группировка по целям кредитования: Недвижимость, Автомобиль, Образование, Свадьба 

### Вывод
#### Для анализа данных требуется:
1. заполнение пропущенных значений
2. устранение артефактов
3. группировка по категориям 

Для удобства нужно сократить десятые значения цифр.  

<div class="alert alert-success">

### Комментарий ревьюера

Хорошо. Данные загружены, общая информация получена. 
</div>

### Шаг 2. Предобработка данных

### Обработка пропусков
#### Пропуск в доходах total_income заполню средним значением по типу занятости

In [ ]:
#df = pd.read_csv('/datasets/data.csv') # ископаемое тестов:) Конечно не надо. 

<div class="alert alert-danger">

### Комментарий ревьюера

Таблица ведь на предыдущем шага уже загружена. зачем это делать опять? 
</div>

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.loc[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [ ]:
def non_value(df, value, category):
    for type_unique in df[category].unique():
        df.loc[(df[value].isna())&(df[category] ==type_unique), value] = df.loc[df[category] ==type_unique, value].median()
    return df

df = non_value(df, 'total_income', 'income_type')  # замена пустых значений дохода средним доходом по этому типу занятости

In [ ]:
df.loc[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


<div class="alert alert-success">

### Комментарий ревьюера 2

Отлично. С доходом разобрались...что там дальше?
</div>

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


####  В трудовом стаже 'days_employed': Отрицательные значения. Слишком большие значения. Пропуски

In [ ]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [ ]:
df.loc[df['days_employed'] < 1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [ ]:
df['days_employed'] =  df['days_employed'].apply(lambda x: x if x>0 else x*-1) 
#это поиск отрицательных и замена их на положительные значения

<div class="alert alert-success">

### Комментарий ревьюера 2

Оу. Лямда ?! круто!
</div>

In [ ]:
df.loc[df['days_employed'] < 1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [ ]:
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [ ]:
df['days_employed'] =  df['days_employed'].apply(lambda x: x if x<133000 else x/24) 
#это поиск огромных значений и их деление на 24 часа. Значение 133000 получено от умножения 5537 (эти значения весят до 75%)*24ч.

In [ ]:
df['days_employed'].describe()

count    19351.000000
mean      4641.641176
std       5355.964289
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      18388.949901
Name: days_employed, dtype: float64

In [ ]:
df.loc[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,118514.486412,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,118514.486412,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,172357.950966,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,142594.396847,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,172357.950966,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,строительство жилой недвижимости


In [ ]:
def no_non_value(df, value, category):
    for type_unique in df[category].unique():
        df.loc[(df[value].isna())&(df[category] ==type_unique), value] = df.loc[df[category] ==type_unique, value].median()
    return df

df = no_non_value(df, 'days_employed', 'dob_years')  # замена пустых значений средним по этому возрасту

In [ ]:
df.loc[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [ ]:
df['days_employed'].describe()

count    21525.000000
mean      4629.994398
std       5317.592062
min         24.141633
25%       1007.368820
50%       2170.569747
75%       5489.396756
max      18388.949901
Name: days_employed, dtype: float64

<div class="alert alert-success">

### Комментарий ревьюера 2

Здорово! Пропуски "повержены". Что там с артефактами?
</div>

#### Артефакт (минус 1) в столбце children заполню '1'

In [ ]:
df.loc[df['children'] == -1, 'children'] = 1

In [ ]:
df['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

#### Поиск нулей

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.loc[df['dob_years'] == 0] 
#Возраст 0? не увидел групп с которыми можно сопоставить для заполнения по похожим значениям, остается как есть

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1759.038033,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
20462,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


In [ ]:
df['gender'].unique() # Непонятное значение в одной записи в столбце "пол" = 'XNA'

array(['F', 'M', 'XNA'], dtype=object)

In [ ]:
df.loc[df['gender'] == 'XNA'] # просрочке по этой записи нет. Остается как есть. Величина не влияет на результаты

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


### Вывод

1) заполнены пропуски по данным "доходы" средним значением по типу занятости (как наиболее зависимыми с доходом, из текущих данных). Применена универсальная конструкция по замене среднего с указанием связанного критерия. Данные готовы для анализа

2) данные "Трудовой стаж": выявленое множество отрицательных значений заменено на положительные значения (из предположения, что это ошибка знака). Выявлена группа некорректных значений в верхнем диапазоне максимума - произведено деление на 24 часа (из предположения, что данные указаны не годах, а в часах. Пропуски заполнены средним значением по возрасту (как наиболее зависимыми со стажем, из текущих данных). Применена универсальная конструкция по замене среднего с указанием связанного критерия. 

3) артефакт (значение минус 1) по данным "Дети" исправлено на положительное значение. Предполагаю, что это был лишний знак.

4) Выявлены нулевые значения  dob_years' Возраст 0? не увидел групп с которыми можно сопоставить для заполнения по похожим значениям, остается как есть. Обратить внимание.

5) Непонятное значение в одной записи в столбце "пол" = 'XNA'. просрочке по этой записи нет. Остается как есть. Величина не влияет на результаты.

  Данные готовы для анализа по заданным критериям.


<div class="alert alert-danger">

### Комментарий ревьюера

Ок, не переживай, тут есть что поправить:
- пропуски в опыте: можно заполнить нулем/единицей или одной медианой/средним, да, но в учебных целях было бы лучше обработать более тщательно. Тем более там нет ничего страшного. Проверь на наличие минусов.  Затем следует обратить внимание на черезчур большие значения в большом количестве строк - при пересчете на года они дают 800-1100 лет опыта (просто поделив на 365!), что неприемлемо. Тогда возможно это и не дни вовсе (не все, а только эти "великаны")?! Возможно достаточно просто поделить большие значения на 24 и все, данные (имеющиеся) будут готовы (обработаны). А насчет заполнения пропусков: вот теперь, когда все имеющиеся значения корректны - можно будет и подумать чем заполнить пропуски... тут можно кастомизировать заполнение пропусков , например, по возрату или по типу занятости. Просто если заполнить пропуски одной общей медианой (средним), то появятся заемщики у которых опыт больше возраста. Но это уже так...рассуждения )
    
- пропуски в доходе: заполнить единицей или одним общим средним/медианой было бы допустимо, если данные не содержили бы больше никаких других полей по которым можно разделить заемщиков. Как ты думаешь предприниматели и студенты будут иметь одинаковый средний/медианный доход? ;)
- помимо явных пропусков таблица еще может содержать неявные - нули. На них тоже следует обратить внимание. ( это не означает что их обязательно как-то обрабатывать или удалять, главное - найти)
- артефакт "-1" в количестве детей - может это просто лишний знак "тире" и по факту это "1".
- в столбце с полом тоже есть кое-какой артефакт: найдем? ;)
- вывод - это место где необходимо не только описывать проделанную работу, но и излагать собственные мысли - почему именно так было выполнено заполнение пропусков? были ли еще варианты? чем таблица теперь лучше?
    
</div>

<div class="alert alert-success">

### Комментарий ревьюера 2

Теперь все очень здорово
</div>

### Замена типа данных

In [ ]:
'Проверим типы данных анализируемых значений', df.dtypes

('Проверим типы данных анализируемых значений',
 children              int64
 days_employed       float64
 dob_years             int64
 education            object
 education_id          int64
 family_status        object
 family_status_id      int64
 gender               object
 income_type          object
 debt                  int64
 total_income        float64
 purpose              object
 dtype: object)

In [ ]:
df['days_employed'].head()

0     8437.673028
1     4024.803754
2     5623.422610
3     4124.747207
4    14177.753002
Name: days_employed, dtype: float64

In [ ]:
'Десятые значения не влияют на значение и мешают восприятию данных. Отражу только целые значения заменив тип float64 на int64',
df['days_employed'] = df['days_employed'].astype(int)

In [ ]:
df['days_employed'].head()

0     8437
1     4024
2     5623
3     4124
4    14177
Name: days_employed, dtype: int64

In [ ]:
df['total_income'].head()

0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
4    158616.077870
Name: total_income, dtype: float64

In [ ]:
'Десятые значения не влияют на анализ и мешают восприятию данных. Отражу только целые значения заменив тип float64 на int64',
df['total_income'] = df['total_income'].astype(int)

In [ ]:
df['total_income'].head()

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод


#### Выявленные дробные значения, которые не влияют на точность анализа и мешают восприятию данных.

Произведена замена типа хранения с float64 на int64 данных total_income, days_employed  
Теперь отражены только целые значения. 

<div class="alert alert-danger">

### Комментарий ревьюера

- опыт исходя легенде проекта - в днях. нужны ли нам там десятые доли?
- доход: у нас нет информации о валюте и стране, т.е. это могут быть хоть индийский рупии. но тем не менее 100500 знаков после запятой тут лишние. 
</div>

<div class="alert alert-success">

### Комментарий ревьюера 2

Готово
</div>

### Обработка дубликатов

In [ ]:
'дубликаты в данных "образование":', df['education'].value_counts()

('дубликаты в данных "образование":',
 среднее                13750
 высшее                  4718
 СРЕДНЕЕ                  772
 Среднее                  711
 неоконченное высшее      668
 ВЫСШЕЕ                   274
 Высшее                   268
 начальное                250
 Неоконченное высшее       47
 НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
 НАЧАЛЬНОЕ                 17
 Начальное                 15
 ученая степень             4
 УЧЕНАЯ СТЕПЕНЬ             1
 Ученая степень             1
 Name: education, dtype: int64)

In [ ]:
df['education'] = df['education'].str.lower()

In [ ]:
'Сделана замена дубликатов в данных Образование', df['education'].value_counts()

('Сделана замена дубликатов в данных Образование',
 среднее                15233
 высшее                  5260
 неоконченное высшее      744
 начальное                282
 ученая степень             6
 Name: education, dtype: int64)

### Вывод
Данные дублируются за счет различных регистров.

Все написания приведены в нижний регистр (с заменой), за счет этого проивзедена четкая группировка. 



<div class="alert alert-danger">

### Комментарий ревьюера

Обработка дубликатов подразумевает работу с полными дублями строк таблицы (удаление их). Поэтому предлагаю в данном пункте найти такие и удалить. И не забудь про регистры перед этим
</div>

In [ ]:
df.duplicated().sum()

71

<div class="alert alert-danger">

### Комментарий ревьюера 2

Так, с регистрами разобрался - молодец. А что насчет непосредственно дублей? имею ввиду полные дубли строк. Они все еще на месте и их следовало бы удалить
</div>

### Лемматизация

In [ ]:
queries = df['purpose'].unique()
queries

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [ ]:
for query in queries:
    if 'жил' in query:
        print(query)

покупка жилья
операции с жильем
покупка жилья для семьи
покупка жилой недвижимости
строительство жилой недвижимости
жилье
покупка своего жилья
покупка жилья для сдачи
ремонт жилью


In [ ]:
for query in queries:
    if 'недв' in query:
        print(query)

покупка недвижимости
покупка коммерческой недвижимости
покупка жилой недвижимости
строительство собственной недвижимости
недвижимость
строительство недвижимости
операции с коммерческой недвижимостью
строительство жилой недвижимости
операции со своей недвижимостью
операции с недвижимостью


In [ ]:
for query in queries:
    if 'автом' in query:
        print(query)

приобретение автомобиля
на покупку подержанного автомобиля
на покупку своего автомобиля
автомобили
сделка с подержанным автомобилем
автомобиль
свой автомобиль
сделка с автомобилем
на покупку автомобиля


In [ ]:
for query in queries:
    if 'рем' in query:
        print(query)

ремонт жилью


In [ ]:
for query in queries:
    if 'образ' in query:
        print(query)

дополнительное образование
образование
заняться образованием
получение образования
получение дополнительного образования
получение высшего образования
профильное образование
высшее образование
заняться высшим образованием


### Вывод

Выявленные составляющие каждой категории цели кредита. Причиной разрозненности данных может быть заполнение цели "со слов заемщика" в дальнейшем целесообразно для уменьшения ошибок и снижения временных затрат на анализ и обработку - формировать базу данных сразу по категориям. 

Не удалось здесь применить lemmatize для выделения слов. Проблема формулировки "ремонт жилью" будет относится к категории "Ремонт" и "Недвижимость". Текущее решение для категоризации сначала замена для категории "Ремонт".

Значение в столбце 'purpose', содержащие:

    'автом' будет заменено в Категоризации на одно слово = 'автомобиль' и так далее:
    'рем' = 'ремонт'
    'свадь' = 'свадьба'
    'жиль' = 'недвижимость'
    'образ' = 'образование'
    'недвиж' = 'недвижимость'
Всего будет четыре категории целей: 'недвижимость', 'ремонт', 'свадьба', 'образование'


<div class="alert alert-success">

### Комментарий ревьюера

Да, здорово. Лемматизация проведена верно
</div>

### Категоризация данных

#### <font color="orange">Отмена:</font>  Замена в данных Дети на две категории 1(да) и 0(нет)

In [ ]:
#df.loc[df['children'] > 0, 'children'] = 1 
#убрал категоризацию по детям

<div class="alert alert-danger">

### Комментарий ревьюера

В начале проекта я уже писал коментарий на этот счет. Не нужно укрупнять хорошие категории.
</div>

#### Группировка данных "Доходы" для формирования групп (категорий)

Чтобы пранализировать зависимость просрочки от дохода, необходимо доходы сгруппировать по диапазонам. До замены пустых значений распределение дохода было 25% = 103 тыс.ед. дохода; 50% =145тыс.ед.; 75% = 203 тыс.руб. Будет 4 группы: 1) До 103 тыс.; 2) До 145 тыс.; 3) До 203 тыс.; 4) Свыше 203 тыс. 

In [ ]:
def group_total_income(x):
    if x < 103000:
        return 'низкий доход <103тыс.ед.'
    if x < 145000:
        return 'средний дохд <145тыс.ед.'
    if x < 203400:
        return 'высокий доход <203тыс.ед.'
    return 'большой доход >203тыс.ед.'

In [ ]:
group_total_income(146756)

'высокий доход <203тыс.ед.'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [ ]:
df['group_total_income'] = df['total_income'].apply(group_total_income)

In [ ]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_total_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,большой доход >203тыс.ед.
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний дохд <145тыс.ед.
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,высокий доход <203тыс.ед.
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,большой доход >203тыс.ед.
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,высокий доход <203тыс.ед.


In [ ]:
df['group_total_income'].value_counts()

средний дохд <145тыс.ед.     6358
высокий доход <203тыс.ед.    5492
большой доход >203тыс.ед.    4841
низкий доход <103тыс.ед.     4834
Name: group_total_income, dtype: int64

<div class="alert alert-success">

### Комментарий ревьюера

На самом деле тут очень качественный и профессиональный подход к выбору диапазонов категоризации (квартили). Супер!... после правильной работы с пропусками в данных по доходу категория "не указали доход" исчезнет" (ну т.е. логика такая: ок что не указали- мы по другим указанным заемщиками признакам уго сформируем)
</div>

#### Категории по целям кредита

In [ ]:
df.loc[df['purpose'].str.contains('автом', regex=False), 'purpose'] = 'автомобиль'
df.loc[df['purpose'].str.contains('рем', regex=False), 'purpose'] = 'ремонт'
df.loc[df['purpose'].str.contains('свадь', regex=False), 'purpose'] = 'свадьба'
df.loc[df['purpose'].str.contains('жиль', regex=False), 'purpose'] = 'недвижимость'
df.loc[df['purpose'].str.contains('образ', regex=False), 'purpose'] = 'образование'
df.loc[df['purpose'].str.contains('недвиж', regex=False), 'purpose'] = 'недвижимость'

In [ ]:
'Сумма по уникальным значениям в целях кредита', df['purpose'].value_counts()

('Сумма по уникальным значениям в целях кредита',
 недвижимость    10228
 автомобиль       4315
 образование      4022
 свадьба          2348
 ремонт            612
 Name: purpose, dtype: int64)

<div class="alert alert-success">

### Комментарий ревьюера

Да, все здорово.
</div>

### Вывод

Категоризация данных "Доход" и "Цели" позволяет провести дальнейший анализ и ответитть на поставленные вопросы .

Чтобы пранализировать зависимость просрочки от дохода, доходы сгруппированы по диапазонам. До замены пустых значений распределение дохода было 25% = 103 тыс.ед. дохода; 50% =145тыс.ед.; 75% = 203 тыс.руб. 

Получено 4 группы с количеством уникальных значений по каждой: 

    средний дохд <145тыс.ед.     6358
    высокий доход <203тыс.ед.    5492
    большой доход >203тыс.ед.    4841
    низкий доход <103тыс.ед.     4834
Данные добавлены в новый столбец.

Количество уникальных значениий по целям кредита: 

     недвижимость    10228
     автомобиль       4315
     образование      4022
     свадьба          2348
     ремонт            612
 
 


<div class="alert alert-warning">

### Комментарий ревьюера

очень рекомендую писать после каждого шага хорошие развернутые выводы
</div>

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
group_children = df.query('children!=20').groupby(['children'])['debt'].agg(['count', 'sum', lambda x: '{:.1%}'.format(x.mean())])
group_children = group_children.reset_index()
group_children.columns = ['Наличие детей', 'Кол-во заемщиков', 'Просрочников', 'Вес просрочников']
group_children

,Наличие детей,Кол-во заемщиков,Просрочников,Вес просрочников
0,0,14149,1063,7.5%
1,1,4865,445,9.1%
2,2,2055,194,9.4%
3,3,330,27,8.2%
4,4,41,4,9.8%
5,5,9,0,0.0%


### Вывод

Наибольшее количество просрочников (1063 человека) не имеют детей, но положительным вяляется наименьший их удельный вес в этой группе.
Самые массовые данные по трем группам "Нет детей", "1 ребенок", "2 детей": по категориям "1 и 2 ребенка" доля просрочников осталяет от 9,1% до 9,3%, но в целом негативным является больший вес просрочников с детьми над просрочниками без детей, более чем на 2%. Такми образом выявлена зависимость - наличие детей явно увеличивает расходы в семье и снижает ее кредитоспособность.

Группа заемщиков, имеющая 3х детей более кредитоспособна (чем с 1 и 2 детьми), вероятно за счет большей ответственности в целом при планировании семьи.

Группы с просрочников с 4мя и 5тью детьми относительно выборки малы, но отрицательным явлется факт роста по группе "4" доли просроченной задолженности.

Группа заемщиков "20 детей" скорее всего артефакт, который исключен из анализируемой таблицы.

Просрочники без детей находятся ниже среднего значения по всей просрочке 8,1%, а просрочники с детьми (независимо от количества) превышают среднее значение минимум на 1%.

Заемщики с детьми с большей вероятностью не погасят кредит вовремя, чем заемщики без детей. 
 

<div class="alert alert-warning">

### Комментарий ревьюера

Очень класно посчитано. 
- в выводах описал не все исследуемые категории а не только самых плохих/хороших по процентам.
- в выводы было бы не плохо добавить твои собственные мысли по поводу полученных результатов
- в самом начале проекта ты посчитал % просрочки по всем: было бы супер если бы ты в выводы добавил сравнение полученных значений с ним

</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
group_family_status = df.groupby(['family_status'])['debt'].agg(['count', 'sum', lambda x: '{:.1%}'.format(x.mean())])
group_family_status = group_family_status.reset_index()
group_family_status.columns = ['Семейное положение', 'Кол-во заемщиков', 'Просрочников', 'Вес просрочников']
group_family_status

,Семейное положение,Кол-во заемщиков,Просрочников,Вес просрочников
0,Не женат / не замужем,2813,274,9.7%
1,в разводе,1195,85,7.1%
2,вдовец / вдова,960,63,6.6%
3,гражданский брак,4177,388,9.3%
4,женат / замужем,12380,931,7.5%


### Вывод

Самые массовые группы:
1) "женат / замужем" имеют наименьшую вероятность просрочки 7,5% в укрупненной группе. И положительным является, что это значение меньше среднего значения по всем просрочникам 8,1%;
2) "гражданский брак" повышает риск просрочки до 9,3% (вероятно с принятием меньшей ответственности в целом) и негативно превышает среднее значение на 1%;  
3) "Не женат / не замужем" имеет худший по казатель в укрупненной группе 9,7%

Существенно лучше исполняют обязательства заемщики в статусах "вдовец / вдова" и "в разводе" 6,6% и 7,1% соответственно.

Таком образом зависимость просрочки от семейного положения есть, чем более официальные отношения в семье, тем меньше риск просрочки. 


<div class="alert alert-warning">

### Комментарий ревьюера

выводы

</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
group_t_income = df.groupby(['group_total_income'])['debt'].agg(['count', 'sum', lambda x: '{:.1%}'.format(x.mean())])
group_t_income = group_t_income.reset_index()
group_t_income.columns = ['Доход', 'Кол-во заемщиков', 'Просрочников', 'Вес просрочников']
group_t_income

,Доход,Кол-во заемщиков,Просрочников,Вес просрочников
0,большой доход >203тыс.ед.,4841,341,7.0%
1,высокий доход <203тыс.ед.,5492,463,8.4%
2,низкий доход <103тыс.ед.,4834,383,7.9%
3,средний дохд <145тыс.ед.,6358,554,8.7%


### Вывод

При относительно ровном распределении заемщиков по группам можно сделать вывод, что наибольшая кредитоспособность у группы с наибольшим доходом свыше 203тыс. (вес просрочки 7%, что на 1,1% лучше среднего показателя 8,1% по всем просрочникам). Вероятно бОльшее количество дохода позволяет лучше обслуживать кредиты.
Далее в порядке убывания рейтинга: группа с низким доходом до 103тыс., 7,9%, но в пределах среднего значения. Вероятно эта группа лучше прогнозирует соотношение доходы/расходы.

Группы со средним (8,7%) и высоким (8,4%) доходом негативно превышают среднее значение по просрочке.

Зависимость дохода и своевременного погашения есть и чем выше или ниже доход, тем меньше вероятность просрочки.

<div class="alert alert-warning">

### Комментарий ревьюера

выводы

</div>

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_group = df.groupby(['purpose'])['debt'].agg(['count', 'sum', lambda x: '{:.1%}'.format(x.mean())])
purpose_group = purpose_group.reset_index()
purpose_group.columns = ['Цели кредита', 'Кол-во по целям', 'Кол-во с просрочкой', 'Вес просрочников']
purpose_group

,Цели кредита,Кол-во по целям,Кол-во с просрочкой,Вес просрочников
0,автомобиль,4315,403,9.3%
1,недвижимость,10228,747,7.3%
2,образование,4022,370,9.2%
3,ремонт,612,35,5.7%
4,свадьба,2348,186,7.9%


### Вывод

Наибольшее количество кредитов взято на приобретение недвижимости и имеет уровень просрочки 7,3%, что меньше среднего значения 8,1%, что является положительным моментом. Вероятно на улучшение качества исполенния обязательств по этой группе играет факт залога приобретаемого жилья.
Наилучшим показателем по исполнению обязательств является группа с целью "Ремонт жилья" 5,7%, но она явлется самой маленькой по количеству выданных кредитов (612 раз). На показатель качества вероятно работает самая "практичная" статья расходов. 

В пределах средней величины группа с целью "свадьба" 7,9%, если проанализировать сроки кредита по договору и по факту погашения, то, вероятно, можно понять источники погашения. 

Худшие показатели кредитов с целями "Образование" (9,2%) и "Автомобиль" (9,3%). Оба превышают средний показатель просрочки 8,1% более чем на 1%. Увеличение риска по образованию вероятно связано с тем, что не могут устроится на работы с требуемым доходом после учебы. Показатель самой рисковой группы можно обосновать, тем, что автомобиль единственный демонстрируемый показатель статуса и его есть желание регулярно завышать. 

Выявлена зависимость уровня просрочки от целей кредита.


<div class="alert alert-warning">

### Комментарий ревьюера

выводы

</div>

### Шаг 4. Общий вывод

#### Цель: Провести анализ данных банка — статистики о платёжеспособности заемщиков, для построения модели кредитного скоринга —  системы, которая позволит экстраполировать фактические данные на способность потенциального заёмщика (со схожими характеристиками) вернуть кредит банку. 

    Данные представлены в CSV формате и содержат 21525 строк в 12 столбцах:
    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

#### По результатам анализа необходимо выявить: 

а) Есть ли зависимость между наличием детей и возвратом кредита в срок?
б) Есть ли зависимость между семейным положением и возвратом кредита в срок?
в) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
г) Как разные цели кредита влияют на его возврат в срок?

#### Для повышения качества анализа проведена оценка представленных данных.
Выявленные некорректные данные прошли следующую предобработку:  

1) заполнены пропуски по данным "доходы" средним значением по типу занятости (как наиболее зависимыми с доходом, из текущих данных). Применена универсальная конструкция по замене среднего с указанием связанного критерия "Тип занятости". 

2) данные "Трудовой стаж": выявленное множество отрицательных значений заменено на положительные значения (из предположения, что это ошибка знака). Выявлена группа некорректных значений в верхнем диапазоне максимума - произведено деление на 24 часа (из предположения, что данные указаны не в днях, а в часах. Пропуски заполнены средним значением по возрасту (как наиболее зависимыми со стажем, из текущих данных). Применена универсальная конструкция по замене среднего с указанием связанного критерия.

3) артефакт (значение минус 1) по данным "Дети" исправлено на положительное значение. Предполагаю, что это был лишний знак.

4) Выявлены нулевые значения dob_years' Нет групп с которыми можно сопоставить для заполнения по похожим значениям, остается как есть. Обратить внимание.

5) Выявленные дробные значения, которые не влияют на точность анализа и мешают восприятию данных. Произведена замена типа хранения с float64 на int64 данных total_income, days_employed.

6) Сделана замена дубликатов в данных "Образование" ('education'). Данные дублируются за счет различных регистров. Все написания приведены в нижний регистр (с заменой), за счет этого произведена четкая группировка.

7) Для группировки по целям кредита устранена разрозненность написания одинаковой цели кредита разным набором слов. Выявленные составляющие каждой категории цели кредита. Причиной разрозненности данных может быть заполнение цели "со слов заемщика". В итоге четыре категории целей: 'недвижимость', 'ремонт', 'свадьба', 'образование'.

8) Данные "Доходы". Чтобы проанализировать зависимость просрочки от дохода, доходы сгруппированы по диапазонам.:
    средний доход <145тыс.ед.     6358 заемщиков
    высокий доход <203тыс.ед.    5492
    большой доход >203тыс.ед.    4841
    низкий доход <103тыс.ед.     4834


#### Зависимость между наличием детей и возвратом кредита в срок
Заемщики с детьми с большей вероятностью не погасят кредит вовремя, чем заемщики без детей.

Просрочники без детей находятся ниже среднего значения по всей просрочке 8,1%, а просрочники с детьми (независимо от количества) превышают среднее значение минимум на 1%.

Наибольшее количество просрочников (1063 человека) не имеют детей, но положительным является наименьший их удельный вес в этой группе. Самые массовые данные по трем группам "Нет детей", "1 ребенок", "2 детей": по категориям "1 и 2 ребенка" доля просрочников состаляет от 9,1% до 9,3%, но в целом негативным является больший вес просрочников с детьми над просрочниками без детей, более чем на 2%. Таким образом выявлена зависимость - наличие детей явно увеличивает расходы в семье и снижает ее кредитоспособность.

Группа заемщиков, имеющая 3х детей более кредитоспособна (чем с 1 и 2 детьми), вероятно за счет большей ответственности в целом при планировании семьи.

Группы с просрочников с 4мя и 5тью детьми относительно выборки малы, но отрицательным является факт роста по группе "4" доли просроченной задолженности.

#### Зависимость между семейным положением и возвратом кредита в срок
Зависимость просрочки от семейного положения есть, чем более официальные отношения в семье, тем меньше риск просрочки.
Самые массовые группы: 1) "женат / замужем" имеют наименьшую вероятность просрочки 7,5% в укрупненной группе. И положительным является, что это значение меньше среднего значения по всем просрочникам 8,1%; 2) "гражданский брак" повышает риск просрочки до 9,3% (вероятно с принятием меньшей ответственности в целом) и негативно превышает среднее значение на 1%; 3) "Не женат / не замужем" имеет худший показатель в укрупненной группе 9,7%

#### Зависимость между уровнем дохода и возвратом кредита в срок
Зависимость дохода и своевременного погашения есть и чем выше или ниже доход, тем меньше вероятность просрочки.
При относительно ровном распределении заемщиков по группам можно сделать вывод, что наибольшая кредитоспособность у группы с наибольшим доходом свыше 203тыс. (вес просрочки 7%, что на 1,1% лучше среднего показателя 8,1% по всем просрочникам). Вероятно бОльшее количество дохода позволяет лучше обслуживать кредиты. Далее в порядке убывания рейтинга: группа с низким доходом до 103тыс., 7,9%, но в пределах среднего значения. Вероятно эта группа лучше прогнозирует соотношение доходы/расходы.

Группы со средним (8,7%) и высоким (8,4%) доходом негативно превышают среднее значение по просрочке.

#### Как разные цели кредита влияют на его возврат в срок
Выявлена зависимость уровня просрочки от целей кредита.
Наибольшее количество кредитов взято на приобретение недвижимости и имеет уровень просрочки 7,3%, что меньше среднего значения 8,1%, что является положительным моментом. Вероятно на улучшение качества исполнения обязательств по этой группе играет факт залога приобретаемого жилья. Наилучшим показателем по исполнению обязательств является группа с целью "Ремонт жилья" 5,7%, но она является самой маленькой по количеству выданных кредитов (612 раз). На показатель качества вероятно работает самая "практичная" статья расходов.

В пределах средней величины группа с целью "свадьба" 7,9%, если проанализировать сроки кредита по договору и по факту погашения, то, вероятно, можно понять источники погашения.

Худшие показатели кредитов с целями "Образование" (9,2%) и "Автомобиль" (9,3%). Оба превышают средний показатель просрочки 8,1% более чем на 1%. Увеличение риска по образованию вероятно связано с тем, что не могут устроится на работы с требуемым доходом после учебы. Показатель самой рисковой группы можно обосновать, тем, что автомобиль единственный демонстрируемый показатель статуса и его есть желание регулярно завышать.

#### В итоге имеются точные данные выявленных характеристик заемщиков при которых понижается или повышается вероятность просрочки. Результаты анализа позволят разработать критерии скоринга и снизить риски банков при кредитовании.



<div class="alert alert-danger">

### Комментарий ревьюера

Полноценного итогового вывода, к сожалению, не обнаружено. В выводах хотелось бы видеть полное описание проекта и полезную информацию полученную в результате работы с данными. Теперь по его наполнению:

- какая перед нами стояла цель?
- какие данные были в нашем распоряжении?
- как мы их обрабатывали/видоизменяли/работали с пропусками + рассуждения на тему появления пропусков/артефактов. Тут очень помогут твои промежуточные выводы.
- общие цифры по всей таблице. Средний процент просрочки, размеры групп и т.п.
- максимально подробное описание полученных зависимостей с рассуждениями.
- и в самом конце "какая цель стояла и выводы помогут эту цель достичь"

Это примерный план итогово вывода к любому аналитическому проекту. Заказчики в реальной работе очень редко смотрят сам проект, а читают только итоговый вывод (они не будут вводить сомостоятельно никакие значения и не будут запускать код). Поэтому так важно делать его максимально подробным и понятным. Предлагаю согласно плану выше изменить свой итоговый вывод и все будет отлично! 
</div>

<div class="alert alert-success">

### Комментарий ревьюера 2

Да, вот это выводище!)
</div>

<div class="alert alert-warning">

### Комментарий ревьюера 2

Не смотря на то, что в проекте остались не удаленными дубликаты, я принимаю проект. Всего 71 дубль - что намного меньше чем 1% от данных - значит они практически не искажают наши выводы. Но постарайся впредь быть внимательнее.
</div>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.